<a href="https://colab.research.google.com/github/njagaty/njagaty3/blob/main/Prediction_of_stock_price_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('GOOG', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", 
             "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3441,2023-01-10 00:00:00-05:00,86.720001,89.474998,86.699997,89.239998,89.239998,22855600
3442,2023-01-11 00:00:00-05:00,90.059998,92.449997,89.739998,92.260002,92.260002,25998800
3443,2023-01-12 00:00:00-05:00,92.400002,92.620003,90.570000,91.910004,91.910004,22754200
3444,2023-01-13 00:00:00-05:00,91.528000,92.980003,90.930000,92.800003,92.800003,18617800
3445,2023-01-17 00:00:00-05:00,92.779999,92.970001,90.839996,92.160004,92.160004,22924800


In [6]:
!pip install yfinance


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 49.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [10]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Google Stock Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

In [11]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    1.000000
Low          0.999875
High         0.999861
Open         0.999714
Volume      -0.511127
Name: Close, dtype: float64


In [13]:
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=30)

Epoch 1/30
2756/2756 [==============================] - 20s 6ms/step - loss: 114.6747
Epoch 2/30
2756/2756 [==============================] - 18s 6ms/step - loss: 5.5665
Epoch 3/30
2756/2756 [==============================] - 17s 6ms/step - loss: 8.1433
Epoch 4/30
2756/2756 [==============================] - 16s 6ms/step - loss: 4.8888
Epoch 5/30
2756/2756 [==============================] - 16s 6ms/step - loss: 6.2012
Epoch 6/30
2756/2756 [==============================] - 16s 6ms/step - loss: 4.7306
Epoch 7/30
2756/2756 [==============================] - 16s 6ms/step - loss: 5.1791
Epoch 8/30
2756/2756 [==============================] - 16s 6ms/step - loss: 3.5582
Epoch 9/30
2756/2756 [==============================] - 16s 6ms/step - loss: 3.0894
Epoch 10/30
2756/2756 [==============================] - 16s 6ms/step - loss: 3.0927
Epoch 11/30
2756/2756 [==============================] - 16s 6ms/step - loss: 3.2666
Epoch 12/30
2756/2756 [==============================] - 16s 6ms/step - 

In [16]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[177.089996, 180.419998, 177.070007, 74919600]])
model.predict(features)

1/1 [==============================] - 1s 947ms/step


array([[149.40926]], dtype=float32)